# Using scaling tool in global model with SSP4 scenario

In [1]:
import pandas as pd
import ixmp
import message_ix
import os

from message_ix.tools.lp_diag import LPdiag
from message_ix.tools.make_scaler import make_scaler
from message_ix.tools.make_scaler import show_range
from message_ix.tools.make_scaler import get_scaler_args

from message_ix.util import make_df

%matplotlib inline

lp = LPdiag()
mp = ixmp.Platform()

## Making a clone of the existing scenario 'baseline'

## Time to Solve the Model

Solution time: 48.14s

Using the unscaled scenario above, we make scaler arguments to solve scenario with scaled matrix coefficients

In [7]:
ssp = "SSP4"
model_name = f"ScalingSSP_{ssp}_v1.0"
scenario_name = f"baseline_DEFAULT"
scen = message_ix.Scenario(mp, model=model_name, scenario=scenario_name)


Existing index sets of 'EMISSION_EQUIVALENCE' ['node', 'emission', 'type_tec', 'year'] do not match []


In [8]:
current_directory = os.getcwd()
mps_dir = os.path.join(current_directory, f"{model_name}_{scenario_name}.mps")
scale_df = make_scaler(mps_dir, scen, bounds=4, steps=1)


Reading MPS-format file C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_scaler\ScalingSSP_SSP4_v1.0_baseline_DEFAULT.mps.
Next section found: NAME          gamsmodel (line 1).
	Problem name: gamsmodel.
Next section found: ROWS (line 2).
	Row _obj (row_seq = 0) is the objective (goal function) row.
Next section found: COLUMNS (line 1230150).
Next section found: RHS (line 7058305).
	Id of RHS: rhs
Next section found: BOUNDS (line 7124408).
	Id of BOUNDS: bnd
Next section found: ENDATA (line 7660942).

Finished processing 7660942 lines of the MPS file: C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\tools\make_scaler\ScalingSSP_SSP4_v1.0_baseline_DEFAULT.mps.
LP has: 1230147 rows, 1136810 cols, 5828154 non-zeros, matrix density = 4.17e-06.
Numbers of redefined: RHS = 66102, ranges = 0, bounds = 536533.

The GF (objective) row named "_obj" has 169 elements.
Distribution of the GF (objective) values:
count    169.000000
mean       0.047908
std

## Run scenarios with scaled matrix

Results: solution time 131.45s

In [ ]:
sc_name = scenario_name+"-scaled"
sc = scen.clone(
    model_name,
    sc_name,
    "solve scenario with scaler",
    keep_solution=False,
)
sc.check_out()

sc.commit(comment="solve scenario with scaler")
sc.set_as_default()

In [ ]:
argument = get_scaler_args(scen)

sc.solve(gams_args=[argument])

In [ ]:
sc.var("OBJ")["lvl"]

In [ ]:
scen.var("OBJ")["lvl"]

**Scaled with default GAMS options:**

Solution time: 136.92s

In [ ]:
sc1_name = scenario_name+"-scaled+defaultgams"
sc1 = scen.clone(
    model_name,
    sc1_name,
    "unscaled daccs scenario with split infrastructure",
    keep_solution=False,
)
sc1.check_out()


argument = get_scaler_args(scen)


sc1.solve(solve_options={"advind": "determined by GAMS Bratio",
                         "epopt": "1.0e-06",
                         "lpmethod": "0",
                         "scaind": "0",
                         },
           gams_args=[argument])
sc1.var("OBJ")["lvl"]

**Unscaled with default GAMS settings**

Solution time: 121.34sec

In [11]:
sc2_name = scenario_name+"-unscaled+defaultgams"
sc2 = scen.clone(
    model_name,
    sc2_name,
    "unscaled daccs scenario with split infrastructure",
    keep_solution=False,
)
sc2.check_out()

sc2.solve(solve_options={"advind": "determined by GAMS Bratio",
                         "epopt": "1.0e-06",
                         "lpmethod": "0",
                         "scaind": "0",
                         })
sc2.var("OBJ")["lvl"]

Existing index sets of 'EMISSION_EQUIVALENCE' ['node', 'emission', 'type_tec', 'year'] do not match []


2479592.5

## Close the connection to the database

In [ ]:
mp.close_db()